In [ ]:
"""
The way the new evaluation works is following this simple algorithm:

if (sentence contains key words):
  We want a compact/'specific' explanation, where only the key words are highlighted
else:
  We want a general/'nuanced' explanations, where all words have similar relevance


How to we check for key words:
 - fidelity: words that flip the prediction are more likely to be key words
 - realism: words that are similar to a set of pre-identified key words
"""

In [ ]:
import numpy as np
import copy
from nltk.corpus import wordnet as wn

In [ ]:
sentence = [""]
x = [0]
model = lambda x: 0
keyWordsClass0 = [] # pre-identified set of key words for class 0
keyWordsClass1 = [] # pre-identified set of key words for class 1

In [ ]:
def realism(sentence, positive_words, negative_words, prediction):
    """
    sentence: a list of words
    positive words: words associated with class 1
    negative words: words associated with class 0
    prediction: the class of the prediction

    returns the indices of the realistic words, if there are any
    """
    keywords = []
    for (i, word) in enumerate(sentence):
        if (prediction == 0):
            for nw in negative_words:
                if (word == nw or wn.synsets(word)[0].wup_similarity(wn.synsets(nw)[0])):
                    keywords.append((i, word))
        if (prediction == 1):
            for pw in positive_words:
                if (word == pw or wn.synsets(word)[0].wup_similarity(wn.synsets(pw)[0])):
                    keywords.append((i, word))
    return keywords

In [ ]:
def fidelity(x, model):
    """
    x: the input words
    model: the prediction model
    """
    keywords = []
    samples = 5
    for (i, w) in enumerate(x):
        if (w != 0):
            change_rating = 0
            for change in range(samples):
                new_x = copy.deepcopy(x)
                new_x[i] = new_x[i] + (100 * change / samples)
                if (model(np.array([x])) != model(np.array([new_x]))):
                    change_rating += 1/samples

            if (change_rating > samples * 3 / 4):
                keywords.append((i, w))

    return keywords

In [ ]:
keywords = []
keywords += realism(sentence, keyWordsClass1, keyWordsClass0, model(sentence))
keywords += fidelity(x, model)